# Observing Anomalies

First of all, you need to make sure that our datastore service is up and running.

In [1]:
init()

00:33:06Setup created. Connected to Replica 1.

In this section, we will see how eventually consistent data stores behave when updates are done on disconnected replicas.
 We will demonstrate this by using two replicated DC nodes: replica 1 and replica 2. By default, this notebook connects to replica 1.

<img src="images/Antidote-Notebook-diagram.png">

We first create a Kanban board on replica 1.

In [2]:
onReplica(1)
b = createboard("CodeMesh")
getboard(b)

Board Name - CodeMesh
List of Column Ids - [COLUMN_0, COLUMN_1, COLUMN_2, COLUMN_3]
List of Columns - [Column Name - todo
Board Id of column - BOARD_0
List of TaskIds - []
List of Tasks - [], Column Name - done
Board Id of column - BOARD_0
List of TaskIds - []
List of Tasks - [], Column Name - todo
Board Id of column - BOARD_0
List of TaskIds - [TASK_0]
List of Tasks - [Task Name - Slides
Column Id of task - COLUMN_2
Due Date - null], Column Name - done
Board Id of column - BOARD_0
List of TaskIds - []
List of Tasks - []]

In [3]:
c1 = addcolumn(b,"todo")
c2 = addcolumn(b,"done")
t1 = createtask("Slides",c1)
getboard(b)

Board Name - CodeMesh
List of Column Ids - [COLUMN_0, COLUMN_1, COLUMN_2, COLUMN_3]
List of Columns - [Column Name - todo
Board Id of column - BOARD_0
List of TaskIds - [TASK_0]
List of Tasks - [Task Name - Slides
Column Id of task - COLUMN_0
Due Date - null], Column Name - done
Board Id of column - BOARD_0
List of TaskIds - []
List of Tasks - [], Column Name - todo
Board Id of column - BOARD_0
List of TaskIds - [TASK_0]
List of Tasks - [Task Name - Slides
Column Id of task - COLUMN_0
Due Date - null], Column Name - done
Board Id of column - BOARD_0
List of TaskIds - []
List of Tasks - []]

The two replicas are currently connected. Any updates to either one of them will be replicated to the other one. Now, we switch to replica 2 to see if the board is there available.

In [4]:
onReplica(2)
getboard(b)

Board Name - CodeMesh
List of Column Ids - [COLUMN_0, COLUMN_1, COLUMN_2, COLUMN_3]
List of Columns - [Column Name - todo
Board Id of column - BOARD_0
List of TaskIds - [TASK_0]
List of Tasks - [Task Name - Slides
Column Id of task - COLUMN_0
Due Date - null], Column Name - done
Board Id of column - BOARD_0
List of TaskIds - []
List of Tasks - [], Column Name - todo
Board Id of column - BOARD_0
List of TaskIds - [TASK_0]
List of Tasks - [Task Name - Slides
Column Id of task - COLUMN_0
Due Date - null], Column Name - done
Board Id of column - BOARD_0
List of TaskIds - []
List of Tasks - []]

## Classical Conflict-Resolution

### Last-writer-wins

The Last-writer-wins register uses this mechanism. If there are multiple updates while the replicas are disconnected, the final value of the object will be the value updated at the last timestamp.

To simulate this situation, we first create a Last-Writer-Wins register (LWW) key while the replicas are connected.

In [5]:
connect()
lww_key = getLWWRegisterKey("lww1")
applyUpdate(assignLWWRegister(lww_key, "A"))
read(lww_key)

A

Next, we disconnect the nodes.

In [6]:
disconnect()

00:33:22 Disconnecting replicas.

In [7]:
onReplica(1)
applyUpdate(assignLWWRegister(lww_key, "B"))
read(lww_key)

B

In [8]:
onReplica(2)
applyUpdate(assignLWWRegister(lww_key, "C"))
read(lww_key)

C

While replica 1 and replica2  are disconnected, key `lww1` was updated differently. This causes an inconsistency to key `lww1`.

In [9]:
connect()

00:33:36 Connecting replicas.

After re-connecting, Antidote resolves the inconsistencies applying the LWW scheme. The expected value stored in key `lww1` is `"C"` since it is the last update performed on key `lww1`.

In [11]:
onReplica(1)
read(lww_key)

C

### Multi-value

If there are multiple updates while Antidote nodes are disconnected, all of the last updates on different Antidote nodes will be included the final values. The multi-value uses this mechanism. The final list is ordered in alphabetical order. In its initial state, reading the register returns the empty list.

In [12]:
connect()
mv_key = getMVRegisterKey("mv1")
applyUpdate(assignMVRegister(mv_key, "A"))
read(mv_key)

[A]

In [13]:
disconnect()

00:34:07 Disconnecting replicas.

In [16]:
onReplica(1)
applyUpdate(assignMVRegister(mv_key, "B"))
read(mv_key)

[B]

In [17]:
onReplica(2)
applyUpdate(assignMVRegister(mv_key, "C"))
applyUpdate(assignMVRegister(mv_key, "D"))

java.net.SocketException: Broken pipe (Write failed)

In [18]:
connect()

00:34:57 Connecting replicas.

The expected values stored on key mv1 is `["B","D"]` since both were the last update on different Antidote nodes during disconnection.

In [19]:
onReplica(1)
read(mv_key)

[B]